In [1]:
pip install nest-asyncio


Note: you may need to restart the kernel to use updated packages.


In [2]:

import nest_asyncio
nest_asyncio.apply()

# Then your original code will work
if __name__ == "__main__":
    import uvicorn
    print("🚀 Starting Multi-Agent AI System Demo Server...")
    print("📍 Visit http://localhost:8000 for the demo interface")
    print("📋 API Documentation: http://localhost:8000/docs")
    uvicorn.run(app, host="0.0.0.0", port=8000)

🚀 Starting Multi-Agent AI System Demo Server...
📍 Visit http://localhost:8000 for the demo interface
📋 API Documentation: http://localhost:8000/docs


NameError: name 'app' is not defined

In [ ]:
from fastapi import FastAPI, File, UploadFile, Form, HTTPException
from fastapi.responses import HTMLResponse, JSONResponse
from pydantic import BaseModel
import json
import os
import sys
from pathlib import Path
from typing import Optional, Dict, Any

# Add the current directory to Python path to import your multi_agent_system module
current_dir = Path("multi_agent_system.ipynb").parent
sys.path.append(str(current_dir))

# Now import your multi-agent system
try:
    from multi_agent_system import MultiAgentSystem, ProcessingResult
except ImportError as e:
    print(f"Error importing multi_agent_system: {e}")
    print("Make sure multi_agent_system.py is in the same directory as this file")
    sys.exit(1)

app = FastAPI(title="Multi-Agent AI System Demo", version="1.0.0")

# Initialize the multi-agent system
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "9o0dvHP8hByOQIcJqdSTmk5JFR5wksEyfshPyj1M9LjLNQLUpZVFwCN3SwV1NxmlN5lXRbPzRVT3BlbkFJqtB0_oVZMtf3vstRBFfbAFKd6r8fo91wt5abRe1f4FeSsDlmGlIQ0Z70Hl9H3XezAZstQ32BAA")
if OPENAI_API_KEY == "":
    print("⚠️  Warning: Please set your OPENAI_API_KEY environment variable")

system = MultiAgentSystem(OPENAI_API_KEY)

class JSONInput(BaseModel):
    data: Dict[Any, Any]
    source_id: Optional[str] = None

class EmailInput(BaseModel):
    content: str
    source_id: Optional[str] = None

@app.get("/", response_class=HTMLResponse)
async def demo_interface():
    """Demo web interface"""
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Multi-Agent AI System Demo</title>
        <style>
            body { font-family: Arial, sans-serif; margin: 40px; background: #f5f5f5; }
            .container { max-width: 1200px; margin: 0 auto; background: white; padding: 30px; border-radius: 10px; box-shadow: 0 2px 10px rgba(0,0,0,0.1); }
            .header { text-align: center; color: #333; margin-bottom: 30px; }
            .section { margin: 20px 0; padding: 20px; border: 1px solid #ddd; border-radius: 5px; }
            .upload-area { background: #f9f9f9; padding: 20px; border: 2px dashed #ccc; border-radius: 5px; text-align: center; }
            textarea { width: 100%; height: 150px; margin: 10px 0; padding: 10px; border: 1px solid #ddd; border-radius: 5px; }
            button { background: #007bff; color: white; padding: 10px 20px; border: none; border-radius: 5px; cursor: pointer; margin: 5px; }
            button:hover { background: #0056b3; }
            .result { background: #e9f7ef; padding: 15px; margin: 10px 0; border-radius: 5px; border-left: 4px solid #28a745; }
            .error { background: #f8d7da; padding: 15px; margin: 10px 0; border-radius: 5px; border-left: 4px solid #dc3545; }
            .demo-section { margin: 20px 0; }
            .demo-button { background: #28a745; margin: 5px; }
            .demo-button:hover { background: #1e7e34; }
            .history-section { background: #f8f9fa; padding: 15px; border-radius: 5px; margin: 10px 0; }
            input[type="text"] { width: 300px; padding: 8px; margin: 5px; border: 1px solid #ddd; border-radius: 3px; }
        </style>
    </head>
    <body>
        <div class="container">
            <div class="header">
                <h1>🤖 Multi-Agent AI System Demo</h1>
                <p>Upload PDF, JSON, or Email content for intelligent classification and processing</p>
            </div>
            
            <div class="demo-section">
                <h3>🚀 Quick Demo</h3>
                <button class="demo-button" onclick="runJSONDemo()">Test JSON Processing</button>
                <button class="demo-button" onclick="runEmailDemo()">Test Email Processing</button>
                <button class="demo-button" onclick="runComplaintDemo()">Test Complaint Processing</button>
            </div>
            
            <div class="section">
                <h3>📄 PDF Upload</h3>
                <div class="upload-area">
                    <input type="file" id="pdfFile" accept=".pdf" />
                    <button onclick="processPDF()">Process PDF</button>
                </div>
            </div>
            
            <div class="section">
                <h3>📧 Email Content</h3>
                <textarea id="emailContent" placeholder="Paste email content here...
From: sender@example.com
Subject: Your subject here
                
Your email content..."></textarea>
                <button onclick="processEmail()">Process Email</button>
            </div>
            
            <div class="section">
                <h3>🔧 JSON Data</h3>
                <textarea id="jsonContent" placeholder='{"id": "example", "type": "invoice", "data": {...}}'></textarea>
                <button onclick="processJSON()">Process JSON</button>
            </div>
            
            <div class="section history-section">
                <h3>📊 Processing History</h3>
                <div>
                    <input type="text" id="sourceId" placeholder="Source ID" />
                    <button onclick="getHistory()">Get History by Source ID</button>
                </div>
                <div>
                    <input type="text" id="threadId" placeholder="Thread ID" />
                    <button onclick="getThreadHistory()">Get Thread History</button>
                </div>
            </div>
            
            <div id="results"></div>
        </div>
        
        <script>
            function showResult(data, type = 'success') {
                const resultsDiv = document.getElementById('results');
                const className = type === 'error' ? 'error' : 'result';
                resultsDiv.innerHTML = `
                    <div class="${className}">
                        <h4>Processing Result:</h4>
                        <pre>${JSON.stringify(data, null, 2)}</pre>
                    </div>
                ` + resultsDiv.innerHTML;
            }
            
            async function processPDF() {
                const fileInput = document.getElementById('pdfFile');
                if (!fileInput.files[0]) {
                    alert('Please select a PDF file');
                    return;
                }
                
                const formData = new FormData();
                formData.append('file', fileInput.files[0]);
                
                try {
                    const response = await fetch('/process/pdf', {
                        method: 'POST',
                        body: formData
                    });
                    const result = await response.json();
                    showResult(result);
                } catch (error) {
                    showResult({error: error.message}, 'error');
                }
            }
            
            async function processEmail() {
                const content = document.getElementById('emailContent').value;
                if (!content.trim()) {
                    alert('Please enter email content');
                    return;
                }
                
                try {
                    const response = await fetch('/process/email', {
                        method: 'POST',
                        headers: {'Content-Type': 'application/json'},
                        body: JSON.stringify({content: content})
                    });
                    const result = await response.json();
                    showResult(result);
                } catch (error) {
                    showResult({error: error.message}, 'error');
                }
            }
            
            async function processJSON() {
                const content = document.getElementById('jsonContent').value;
                if (!content.trim()) {
                    alert('Please enter JSON data');
                    return;
                }
                
                try {
                    const jsonData = JSON.parse(content);
                    const response = await fetch('/process/json', {
                        method: 'POST',
                        headers: {'Content-Type': 'application/json'},
                        body: JSON.stringify({data: jsonData})
                    });
                    const result = await response.json();
                    showResult(result);
                } catch (error) {
                    showResult({error: error.message}, 'error');
                }
            }
            
            async function getHistory() {
                const sourceId = document.getElementById('sourceId').value;
                if (!sourceId.trim()) {
                    alert('Please enter a source ID');
                    return;
                }
                
                try {
                    const response = await fetch(`/history/${sourceId}`);
                    const result = await response.json();
                    showResult(result);
                } catch (error) {
                    showResult({error: error.message}, 'error');
                }
            }
            
            async function getThreadHistory() {
                const threadId = document.getElementById('threadId').value;
                if (!threadId.trim()) {
                    alert('Please enter a thread ID');
                    return;
                }
                
                try {
                    const response = await fetch(`/thread/${threadId}`);
                    const result = await response.json();
                    showResult(result);
                } catch (error) {
                    showResult({error: error.message}, 'error');
                }
            }
            
            // Demo functions
            async function runJSONDemo() {
                const demoJSON = {
                    "id": "INV-2024-001",
                    "type": "invoice",
                    "vendor": "TechCorp Solutions",
                    "amount": 2500.00,
                    "due_date": "2024-02-15",
                    "items": [
                        {"description": "Software License", "quantity": 5, "price": 500}
                    ],
                    "sender": "billing@techcorp.com"
                };
                
                document.getElementById('jsonContent').value = JSON.stringify(demoJSON, null, 2);
                await processJSON();
            }
            
            async function runEmailDemo() {
                const demoEmail = `From: procurement@company.com
Subject: RFQ - Office Supplies Q1 2024

Dear Vendor,

We are requesting quotations for the following office supplies for Q1 2024:

1. A4 Paper - 100 reams
2. Printer Cartridges - HP LaserJet series
3. Office Chairs - Ergonomic, 25 units
4. Desk Organizers - 50 units

Please provide your best pricing and delivery timeline.

Deadline for submission: January 30, 2024

Best regards,
Sarah Johnson
Procurement Manager`;
                
                document.getElementById('emailContent').value = demoEmail;
                await processEmail();
            }
            
            async function runComplaintDemo() {
                const complaintEmail = `From: angry.customer@email.com
Subject: URGENT - Defective Product Complaint

Dear Customer Service,

I am extremely disappointed with my recent purchase (Order #12345). 
The product arrived damaged and completely unusable. This is the third time 
I've had issues with your company's products.

I demand immediate replacement and compensation for the inconvenience.
This is affecting my business operations and I need resolution TODAY.

If this is not resolved within 24 hours, I will be escalating to management
and considering legal action.

Furious Customer
John Smith`;
                
                document.getElementById('emailContent').value = complaintEmail;
                await processEmail();
            }
        </script>
    </body>
    </html>
    """
    return html_content

@app.post("/process/json")
async def process_json_endpoint(input_data: JSONInput):
    """Process JSON input"""
    try:
        result = system.process_input(input_data.data, input_data.source_id)
        return {
            "status": "success",
            "source_id": result.source_id,
            "format": result.format_type.value,
            "intent": result.intent.value,
            "sender": result.sender,
            "timestamp": result.timestamp.isoformat(),
            "extracted_data": result.extracted_data,
            "anomalies": result.anomalies,
            "thread_id": result.thread_id
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/process/email")
async def process_email_endpoint(input_data: EmailInput):
    """Process email input"""
    try:
        result = system.process_input(input_data.content, input_data.source_id)
        return {
            "status": "success",
            "source_id": result.source_id,
            "format": result.format_type.value,
            "intent": result.intent.value,
            "sender": result.sender,
            "timestamp": result.timestamp.isoformat(),
            "extracted_data": result.extracted_data,
            "anomalies": result.anomalies,
            "thread_id": result.thread_id
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/process/pdf")
async def process_pdf_endpoint(file: UploadFile = File(...)):
    """Process PDF input"""
    try:
        if not file.filename.endswith('.pdf'):
            raise HTTPException(status_code=400, detail="File must be a PDF")
        
        content = await file.read()
        result = system.process_input(content, file.filename)
        
        return {
            "status": "success",
            "source_id": result.source_id,
            "format": result.format_type.value,
            "intent": result.intent.value,
            "sender": result.sender,
            "timestamp": result.timestamp.isoformat(),
            "extracted_data": result.extracted_data,
            "anomalies": result.anomalies,
            "thread_id": result.thread_id
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/history/{source_id}")
async def get_processing_history(source_id: str):
    """Get processing history for a source ID"""
    try:
        history = system.get_processing_history(source_id=source_id)
        if not history:
            raise HTTPException(status_code=404, detail="No history found for source ID")
        
        return {
            "status": "success",
            "history": [
                {
                    "source_id": result.source_id,
                    "format": result.format_type.value,
                    "intent": result.intent.value,
                    "sender": result.sender,
                    "timestamp": result.timestamp.isoformat(),
                    "extracted_data": result.extracted_data,
                    "anomalies": result.anomalies,
                    "thread_id": result.thread_id
                } for result in history
            ]
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/thread/{thread_id}")
async def get_thread_history(thread_id: str):
    """Get all processing results for a thread ID"""
    try:
        history = system.get_processing_history(thread_id=thread_id)
        if not history:
            raise HTTPException(status_code=404, detail="No thread history found")
        
        return {
            "status": "success",
            "thread_id": thread_id,
            "message_count": len(history),
            "history": [
                {
                    "source_id": result.source_id,
                    "format": result.format_type.value,
                    "intent": result.intent.value,
                    "sender": result.sender,
                    "timestamp": result.timestamp.isoformat(),
                    "extracted_data": result.extracted_data,
                    "anomalies": result.anomalies
                } for result in history
            ]
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/health")
async def health_check():
    """Health check endpoint"""
    return {"status": "healthy", "message": "Multi-Agent AI System is running"}

@app.get("/stats")
async def get_system_stats():
    """Get system processing statistics"""
    try:
        # You can extend this to get more detailed stats from your database
        return {
            "status": "success",
            "message": "System statistics endpoint - extend as needed",
            "endpoints": {
                "process_json": "/process/json",
                "process_email": "/process/email", 
                "process_pdf": "/process/pdf",
                "get_history": "/history/{source_id}",
                "get_thread": "/thread/{thread_id}",
                "health_check": "/health"
            }
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

if __name__ == "__main__":
    import uvicorn
    print("🚀 Starting Multi-Agent AI System Demo Server...")
    print("📍 Visit http://localhost:8000 for the demo interface")
    print("📋 API Documentation: http://localhost:8000/docs")
    uvicorn.run(app, host="0.0.0.0", port=8000)

✅ API Key set!
🚀 System ready!


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.438631 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.756275 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
ERROR:multi_agent_system:Error in intent classification: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
INFO:multi_agent_system:Classified 32d6ae7a: Format=JSON, Intent=GENERAL
INFO:multi_agent_system:Stored result for 32d6ae7a


SUCCESS: JSON - GENERAL
🧪 Testing main system...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.444057 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.988063 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
ERROR:multi_agent_system:Error in intent classification: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
INFO:multi_agent_system:Classified a537caca: Format=JSON, Intent=GENERAL
INFO:multi_agent_system:Stored result for a537caca


✅ System working: JSON
🚀 SIMPLE INTERFACE LOADED!

📋 Available Functions:
• test_multi_agent_system() - Run all tests
• process_my_json(data) - Test your JSON
• process_my_email('text') - Test your email
• DEMO_DATA - Pre-made test data

✨ Ready to test! Run: test_multi_agent_system()
🤖 MULTI-AGENT SYSTEM TESTER

1️⃣ TESTING JSON PROCESSING...


...

INFO:     127.0.0.1:52709 - "POST /process/email HTTP/1.1" 200 OK


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.490125 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.971709 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
ERROR:multi_agent_system:Error in intent classification: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
INFO:multi_agent_system:Classified 6b43646f: Format=EMAIL, Intent=GENERAL
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "

INFO:     127.0.0.1:52709 - "POST /process/email HTTP/1.1" 200 OK


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.414483 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.846413 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
ERROR:multi_agent_system:Error in intent classification: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
INFO:multi_agent_system:Classified 6b43646f: Format=EMAIL, Intent=GENERAL
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "

INFO:     127.0.0.1:52779 - "POST /process/email HTTP/1.1" 200 OK
